#### import libraries 

In [1]:
import pandas as pd
import pickle
import numpy as np
from datetime import datetime
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder

import warnings
warnings.filterwarnings('ignore')

In [2]:
df = pd.read_excel(r'data\tdatanew.xlsx')
df 



,transaction_id,customer_id,product_id,product_name,category,purchase_date,quantity,price_per_unit,total_amount,payment_method,store_location,Mobile
0,T000290,C0259,P004,Cricket Cap,Cricket,2024-07-23,1,300,300,Debit Card,Kolkata,9400950651
1,T000223,C0226,P019,Shuttlecock,Badminton,2025-03-11,1,150,150,Debit Card,Delhi,9400950651
2,T000213,C0083,P019,Shuttlecock,Badminton,2024-09-06,1,150,150,Debit Card,Delhi,9400950651
3,T000458,C0263,P014,Yoga Mat,Yoga,2025-05-18,1,700,700,Credit Card,Bangalore,9400950651
4,T000308,C0132,P027,Tennis Balls,Tennis,2025-03-09,1,250,250,Debit Card,Kolkata,9400950651
...,...,...,...,...,...,...,...,...,...,...,...,...
2495,T000328,C0023,P015,Yoga Bottle,Yoga,2024-01-04,1,350,350,Debit Card,Mumbai,9400950651
2496,T000220,C0137,P001,Cricket Ball,Cricket,2024-08-30,1,200,200,Debit Card,Delhi,9400950651
2497,T000227,C0100,P020,Badminton Net,Badminton,2024-09-16,3,1200,3600,Cash,Pune,9400950651
2498,T000259,C0234,P005,Football,Football,2024-11-06,1,1200,1200,UPI,Kolkata,9400950651


In [3]:
df['purchase_date'] = pd.to_datetime(df['purchase_date']).dt.date
df['transaction_id'] = df['customer_id'].astype(str)
df


,transaction_id,customer_id,product_id,product_name,category,purchase_date,quantity,price_per_unit,total_amount,payment_method,store_location,Mobile
0,C0259,C0259,P004,Cricket Cap,Cricket,2024-07-23,1,300,300,Debit Card,Kolkata,9400950651
1,C0226,C0226,P019,Shuttlecock,Badminton,2025-03-11,1,150,150,Debit Card,Delhi,9400950651
2,C0083,C0083,P019,Shuttlecock,Badminton,2024-09-06,1,150,150,Debit Card,Delhi,9400950651
3,C0263,C0263,P014,Yoga Mat,Yoga,2025-05-18,1,700,700,Credit Card,Bangalore,9400950651
4,C0132,C0132,P027,Tennis Balls,Tennis,2025-03-09,1,250,250,Debit Card,Kolkata,9400950651
...,...,...,...,...,...,...,...,...,...,...,...,...
2495,C0023,C0023,P015,Yoga Bottle,Yoga,2024-01-04,1,350,350,Debit Card,Mumbai,9400950651
2496,C0137,C0137,P001,Cricket Ball,Cricket,2024-08-30,1,200,200,Debit Card,Delhi,9400950651
2497,C0100,C0100,P020,Badminton Net,Badminton,2024-09-16,3,1200,3600,Cash,Pune,9400950651
2498,C0234,C0234,P005,Football,Football,2024-11-06,1,1200,1200,UPI,Kolkata,9400950651


### Feature engineering 

In [4]:
d1 = df.groupby('customer_id').agg(
    Monetary=('total_amount', 'sum'),
    total_quantity=('quantity', 'sum'),
    Frequency=('transaction_id', 'count'),
    num_unique_products=('product_id', 'nunique'),
    last_purchase_date=('purchase_date', 'max'),
    avg_price_per_unit=('price_per_unit', 'mean'),
    store_visit_frequency=('purchase_date', 'nunique'),
    Mobile = ('Mobile','first')
).reset_index()
d1

,customer_id,Monetary,total_quantity,Frequency,num_unique_products,last_purchase_date,avg_price_per_unit,store_visit_frequency,Mobile
0,C0000,26550,23,15,12,2025-01-07,1023.333333,3,9400950651
1,C0001,5750,6,5,5,2024-05-06,790.000000,1,9400950651
2,C0003,9250,7,5,5,2025-03-17,1130.000000,1,9400950651
3,C0004,8500,6,5,5,2024-04-29,1460.000000,1,9400950651
4,C0006,8250,8,5,5,2024-01-12,1150.000000,1,9400950651
...,...,...,...,...,...,...,...,...,...
282,C0395,12000,8,5,5,2025-05-29,1340.000000,1,9400950651
283,C0396,7350,6,5,5,2024-08-10,1230.000000,1,9400950651
284,C0397,11500,11,10,10,2025-04-07,1030.000000,2,9400950651
285,C0398,7100,9,5,5,2024-04-15,780.000000,1,9400950651


In [5]:
membership_start = df.groupby('customer_id')['purchase_date'].min().reset_index()
membership_start.rename(columns={'purchase_date':'membership_start_date'}, inplace=True)

d1 = d1.merge(membership_start, on='customer_id', how='left')
d1

,customer_id,Monetary,total_quantity,Frequency,num_unique_products,last_purchase_date,avg_price_per_unit,store_visit_frequency,Mobile,membership_start_date
0,C0000,26550,23,15,12,2025-01-07,1023.333333,3,9400950651,2024-05-26
1,C0001,5750,6,5,5,2024-05-06,790.000000,1,9400950651,2024-05-06
2,C0003,9250,7,5,5,2025-03-17,1130.000000,1,9400950651,2025-03-17
3,C0004,8500,6,5,5,2024-04-29,1460.000000,1,9400950651,2024-04-29
4,C0006,8250,8,5,5,2024-01-12,1150.000000,1,9400950651,2024-01-12
...,...,...,...,...,...,...,...,...,...,...
282,C0395,12000,8,5,5,2025-05-29,1340.000000,1,9400950651,2025-05-29
283,C0396,7350,6,5,5,2024-08-10,1230.000000,1,9400950651,2024-08-10
284,C0397,11500,11,10,10,2025-04-07,1030.000000,2,9400950651,2024-04-29
285,C0398,7100,9,5,5,2024-04-15,780.000000,1,9400950651,2024-04-15


In [6]:
#d1.to_excel('group_data.xlsx', index=False)

In [7]:
reference_date = pd.to_datetime(d1['last_purchase_date'].max())
today = pd.to_datetime(datetime.today().date())

d1['membership_start_date'] = pd.to_datetime(d1['membership_start_date'])
d1['last_purchase_date'] = pd.to_datetime(d1['last_purchase_date'])

d1['Active_days'] = ((reference_date - d1['membership_start_date']).dt.days).round().astype(int)
d1['Avg_purchase_gap_days'] = d1.apply(lambda x: x['Active_days'] / x['store_visit_frequency']
                                       if x['store_visit_frequency'] > 0 else x['Active_days'], axis=1)

d1['Recency'] = (today - d1['last_purchase_date']).dt.days
d1

,customer_id,Monetary,total_quantity,Frequency,num_unique_products,last_purchase_date,avg_price_per_unit,store_visit_frequency,Mobile,membership_start_date,Active_days,Avg_purchase_gap_days,Recency
0,C0000,26550,23,15,12,2025-01-07,1023.333333,3,9400950651,2024-05-26,370,123.333333,154
1,C0001,5750,6,5,5,2024-05-06,790.000000,1,9400950651,2024-05-06,390,390.000000,400
2,C0003,9250,7,5,5,2025-03-17,1130.000000,1,9400950651,2025-03-17,75,75.000000,85
3,C0004,8500,6,5,5,2024-04-29,1460.000000,1,9400950651,2024-04-29,397,397.000000,407
4,C0006,8250,8,5,5,2024-01-12,1150.000000,1,9400950651,2024-01-12,505,505.000000,515
...,...,...,...,...,...,...,...,...,...,...,...,...,...
282,C0395,12000,8,5,5,2025-05-29,1340.000000,1,9400950651,2025-05-29,2,2.000000,12
283,C0396,7350,6,5,5,2024-08-10,1230.000000,1,9400950651,2024-08-10,294,294.000000,304
284,C0397,11500,11,10,10,2025-04-07,1030.000000,2,9400950651,2024-04-29,397,198.500000,64
285,C0398,7100,9,5,5,2024-04-15,780.000000,1,9400950651,2024-04-15,411,411.000000,421


In [8]:
features = ['Monetary','Frequency','Recency','Active_days','total_quantity','avg_price_per_unit','store_visit_frequency',
            'Avg_purchase_gap_days']
data = d1[features]

scaler = StandardScaler()
scaled_data = scaler.fit_transform(data)
scaled_data

array([[ 2.45467101,  1.32943594, -0.48617815, ...,  0.3501214 ,
         1.35012357, -0.69761412],
       [-0.75264041, -0.78440403,  1.2486419 , ..., -0.66746133,
        -0.78478257,  1.49046196],
       [-0.21294859, -0.78440403, -0.97277401, ...,  0.81530207,
        -0.78478257, -1.09420291],
       ...,
       [ 0.13399616,  0.27251595, -1.12086841, ...,  0.37919519,
         0.2826705 , -0.08085017],
       [-0.54447357, -0.78440403,  1.39673629, ..., -0.71107202,
        -0.78478257,  1.66277295],
       [ 0.54261997,  0.27251595, -1.04329515, ..., -0.31857582,
         0.2826705 , -0.62650165]])

In [9]:
with open('Models/CS_scalers.pkl','wb') as file:
    pickle.dump(scaler,file)

#### Features

In [10]:
kmeans = KMeans(n_clusters=4, random_state=42)
kmeans.fit(scaled_data)

d1['cluster'] = kmeans.labels_
d1

,customer_id,Monetary,total_quantity,Frequency,num_unique_products,last_purchase_date,avg_price_per_unit,store_visit_frequency,Mobile,membership_start_date,Active_days,Avg_purchase_gap_days,Recency,cluster
0,C0000,26550,23,15,12,2025-01-07,1023.333333,3,9400950651,2024-05-26,370,123.333333,154,1
1,C0001,5750,6,5,5,2024-05-06,790.000000,1,9400950651,2024-05-06,390,390.000000,400,2
2,C0003,9250,7,5,5,2025-03-17,1130.000000,1,9400950651,2025-03-17,75,75.000000,85,0
3,C0004,8500,6,5,5,2024-04-29,1460.000000,1,9400950651,2024-04-29,397,397.000000,407,2
4,C0006,8250,8,5,5,2024-01-12,1150.000000,1,9400950651,2024-01-12,505,505.000000,515,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
282,C0395,12000,8,5,5,2025-05-29,1340.000000,1,9400950651,2025-05-29,2,2.000000,12,0
283,C0396,7350,6,5,5,2024-08-10,1230.000000,1,9400950651,2024-08-10,294,294.000000,304,2
284,C0397,11500,11,10,10,2025-04-07,1030.000000,2,9400950651,2024-04-29,397,198.500000,64,3
285,C0398,7100,9,5,5,2024-04-15,780.000000,1,9400950651,2024-04-15,411,411.000000,421,2


In [11]:
with open('Models/CS_model.pkl', 'wb') as file:
    pickle.dump(kmeans, file)

In [12]:
agg = d1.groupby(['cluster']).agg({
    'Frequency': 'sum',
    'Monetary': 'sum'
}).rename(columns={
    'Frequency': 'Frequency',
    'Monetary': 'Monetary'
}).reset_index()

agg['Unit Price'] = (agg['Monetary'] / agg['Frequency'])
agg



,cluster,Frequency,Monetary,Unit Price
0,0,415,525900,1267.228916
1,1,860,1071300,1245.697674
2,2,375,437850,1167.600000
3,3,850,1016050,1195.352941


In [13]:
tier_labels = ['Platinum', 'Gold', 'Silver', 'Bronze']

agg = agg.sort_values('Unit Price', ascending=False)
agg['loyalty'] = tier_labels[:len(agg)]
agg

,cluster,Frequency,Monetary,Unit Price,loyalty
0,0,415,525900,1267.228916,Platinum
1,1,860,1071300,1245.697674,Gold
3,3,850,1016050,1195.352941,Silver
2,2,375,437850,1167.600000,Bronze


In [14]:
reward_mapping = {
    'Platinum': '25% discount + VIP concierge access',
    'Gold': '20% discount + free shipping',
    'Silver': '15% discount or birthday bonus',
    'Bronze': 'Points-based rewards or 10% discount'}



In [15]:
agg['assigned_reward'] = agg['loyalty'].map(reward_mapping)
agg=agg[['cluster','loyalty','assigned_reward']]

In [16]:
final = d1.merge(agg, on='cluster', how='left')
final

,customer_id,Monetary,total_quantity,Frequency,num_unique_products,last_purchase_date,avg_price_per_unit,store_visit_frequency,Mobile,membership_start_date,Active_days,Avg_purchase_gap_days,Recency,cluster,loyalty,assigned_reward
0,C0000,26550,23,15,12,2025-01-07,1023.333333,3,9400950651,2024-05-26,370,123.333333,154,1,Gold,20% discount + free shipping
1,C0001,5750,6,5,5,2024-05-06,790.000000,1,9400950651,2024-05-06,390,390.000000,400,2,Bronze,Points-based rewards or 10% discount
2,C0003,9250,7,5,5,2025-03-17,1130.000000,1,9400950651,2025-03-17,75,75.000000,85,0,Platinum,25% discount + VIP concierge access
3,C0004,8500,6,5,5,2024-04-29,1460.000000,1,9400950651,2024-04-29,397,397.000000,407,2,Bronze,Points-based rewards or 10% discount
4,C0006,8250,8,5,5,2024-01-12,1150.000000,1,9400950651,2024-01-12,505,505.000000,515,2,Bronze,Points-based rewards or 10% discount
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
282,C0395,12000,8,5,5,2025-05-29,1340.000000,1,9400950651,2025-05-29,2,2.000000,12,0,Platinum,25% discount + VIP concierge access
283,C0396,7350,6,5,5,2024-08-10,1230.000000,1,9400950651,2024-08-10,294,294.000000,304,2,Bronze,Points-based rewards or 10% discount
284,C0397,11500,11,10,10,2025-04-07,1030.000000,2,9400950651,2024-04-29,397,198.500000,64,3,Silver,15% discount or birthday bonus
285,C0398,7100,9,5,5,2024-04-15,780.000000,1,9400950651,2024-04-15,411,411.000000,421,2,Bronze,Points-based rewards or 10% discount


In [18]:
FREQ_THRESHOLD = 15
MONETARY_THRESHOLD = 30000

def apply_reward_rules(row):
    if row['Frequency'] < FREQ_THRESHOLD and row['Monetary'] < MONETARY_THRESHOLD:
        loyalty_tier = "No tier"
        progress_message = (f"⚠️ Hi Customer {row['customer_id']}! You currently have no loyalty tier. "
                            "Shop more to unlock exciting rewards like 20% discount + free shipping. Start today and join the club! 💪")
    else:
        loyalty_tier = row['loyalty']
        purchase_gap = max(0, FREQ_THRESHOLD - row['Frequency'])
        money_gap = max(0, MONETARY_THRESHOLD - row['Monetary'])
        
        if purchase_gap == 0 or money_gap == 0:
            progress_message = (f"🎊 Congrats Customer {row['customer_id']}! As a {loyalty_tier} member, "
                                f"enjoy your reward: {row['assigned_reward']}! Thanks for being awesome! 💖")
        else:
            progress_message = (f"🎉 Hey Customer {row['customer_id']}! You're rocking the {loyalty_tier} tier! "
                                f"Only {purchase_gap} more purchases or ₹{money_gap:.0f} more to unlock your exclusive reward. Keep it up! 🚀")

    return pd.Series([loyalty_tier, row['assigned_reward'], progress_message])

final[['loyalty', 'assigned_reward', 'progress_message']] = final.apply(apply_reward_rules, axis=1)
final.tail()


,customer_id,Monetary,total_quantity,Frequency,num_unique_products,last_purchase_date,avg_price_per_unit,store_visit_frequency,Mobile,membership_start_date,Active_days,Avg_purchase_gap_days,Recency,cluster,loyalty,assigned_reward,progress_message
282,C0395,12000,8,5,5,2025-05-29,1340.0,1,9400950651,2025-05-29,2,2.0,12,0,No tier,25% discount + VIP concierge access,⚠️ Hi Customer C0395! You currently have no lo...
283,C0396,7350,6,5,5,2024-08-10,1230.0,1,9400950651,2024-08-10,294,294.0,304,2,No tier,Points-based rewards or 10% discount,⚠️ Hi Customer C0396! You currently have no lo...
284,C0397,11500,11,10,10,2025-04-07,1030.0,2,9400950651,2024-04-29,397,198.5,64,3,No tier,15% discount or birthday bonus,⚠️ Hi Customer C0397! You currently have no lo...
285,C0398,7100,9,5,5,2024-04-15,780.0,1,9400950651,2024-04-15,411,411.0,421,2,No tier,Points-based rewards or 10% discount,⚠️ Hi Customer C0398! You currently have no lo...
286,C0399,14150,14,10,8,2025-03-27,870.0,2,9400950651,2024-09-09,264,132.0,75,3,No tier,15% discount or birthday bonus,⚠️ Hi Customer C0399! You currently have no lo...


In [57]:
# FREQ_THRESHOLD = 15
# MONETARY_THRESHOLD = 30000

# def apply_reward_rules(row):

#     if row['Frequency'] >= FREQ_THRESHOLD or row['Monetary'] >= MONETARY_THRESHOLD:
#         return pd.Series([row['loyalty'], row['assigned_reward'], "✅ Eligible for reward"])
#     else:
#         purchase_gap = max(0, FREQ_THRESHOLD - row['Frequency'])A
#         money_gap = max(0, MONETARY_THRESHOLD - row['Monetary'])
#         msg = f"🔔 You need {purchase_gap} more purchases or ₹{money_gap:.0f} more to earn a reward."
#         return pd.Series(['No reward', 'No reward', msg])

# final[['loyalty', 'assigned_reward', 'progress_message']] = final.apply(apply_reward_rules, axis=1)


# final

In [27]:
# import pandas as pd

# FREQ_THRESHOLD = 5
# MONETARY_THRESHOLD = 5000

# def assign_rewards_to_new_data(new_data, kmeans_model, scaler, agg):

#     # 1. Scale the new data features (use same scaler as training)
#     features = ['Frequency', 'Monetary', 'Avg_monetary', 'store_visit_frequency', 'Active_days']  # Adjust based on your features
#     scaled_features = scaler.transform(new_data[features])

#     # 2. Predict cluster for new customers
#     new_data['cluster'] = kmeans_model.predict(scaled_features)

#     # 3. Map cluster to loyalty tier and assigned reward
#     new_data = new_data.merge(agg, on='cluster', how='left')

#     # 4. Apply reward eligibility rules
#     def apply_reward_rules(row):
#         if row['Frequency'] >= FREQ_THRESHOLD or row['Monetary'] >= MONETARY_THRESHOLD:
#             return pd.Series([row['loyalty'], row['assigned_reward'], "✅ Eligible for reward"])
#         else:
#             purchase_gap = max(0, FREQ_THRESHOLD - row['Frequency'])
#             money_gap = max(0, MONETARY_THRESHOLD - row['Monetary'])
#             msg = f"🔔 You need {purchase_gap} more purchases or ₹{money_gap:.0f} more to earn a reward."
#             return pd.Series([None, None, msg])

#     new_data[['loyalty', 'assigned_reward', 'progress_message']] = new_data.apply(apply_reward_rules, axis=1)

#     return new_data

